In [1]:
pip install tidytext textblob

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 KB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 KB 34.0 MB/s eta 0:00:00
  Created wheel for tidytext: filename=tidytext-0.0.1-py3-none-any.whl size=3895 sha256=ce0fe0e1887c3e2ee298834c3c589b2feab22ec757eb3d0a86c64a99476dfaf5
  Stored in directory: /home/ec2-user/.cache/pip/wheels/07/03/c0/f73eeef462dd66dbca0288a338fcbcdc78e3588937ccc907d8
Successfully built tidytext
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 200)
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import re
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm import tqdm

from Help_Funs import count_chars, count_words, count_capital_chars, count_capital_words, count_sent, count_unique_words, count_stopwords, count_hashtags 

s3 = boto3.resource('s3')
s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

## Defining files names
file_key_1 = 'NLP-Disaster-Tweets/train.csv'
file_key_2 = 'NLP-Disaster-Tweets/test.csv'
file_key_3 = 'NLP-Disaster-Tweets/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data-files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
sample = pd.read_csv(file_content_stream_3)

In [4]:
## Defining stopwords
stop_words = set(stopwords.words('english'))
others = set(["1", "2", "it'll", "ill", "=", '+', "'s'", '"'])
stop_words = stop_words.union(others)

def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stop_words]
    temp = " ".join(word for word in temp)
    return temp

In [5]:
train['clean_tweet'] = np.nan

for i in tqdm(range(0, train.shape[0])):
    
    train['clean_tweet'][i] =  clean_tweet(train['text'][i])

  0%|          | 0/7613 [00:00<?, ?it/s]/tmp/ipykernel_22854/4248643825.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['clean_tweet'][i] =  clean_tweet(train['text'][i])
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████| 7613/7613 [00:00<00:00, 23668.28it/s]


In [6]:
test['clean_tweet'] = np.nan

for i in tqdm(range(0, test.shape[0])):
    
    test['clean_tweet'][i] =  clean_tweet(test['text'][i])

  0%|          | 0/3263 [00:00<?, ?it/s]/tmp/ipykernel_22854/1846826793.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['clean_tweet'][i] =  clean_tweet(test['text'][i])
100%|██████████| 3263/3263 [00:00<00:00, 30411.00it/s]


In [7]:
train['sentiment'] = train['clean_tweet'].apply(lambda x: TextBlob(x).sentiment.polarity)
train['subjectivity'] = train['clean_tweet'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

test['sentiment'] = test['clean_tweet'].apply(lambda x: TextBlob(x).sentiment.polarity)
test['subjectivity'] = test['clean_tweet'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

In [8]:
train['sentiment_label'] = np.where(train['sentiment'] < 0, 'negative', 
                                    np.where(train['sentiment'] == 0, 'neutral', 'positive'))

test['sentiment_label'] = np.where(test['sentiment'] < 0, 'negative', 
                                   np.where(test['sentiment'] == 0, 'neutral', 'positive'))

In [9]:
train_dummies = pd.get_dummies(train['sentiment_label'])
train = pd.concat([train.drop(columns = ['sentiment', 'sentiment_label'], axis = 1), train_dummies], axis = 1)

test_dummies = pd.get_dummies(test['sentiment_label'])
test = pd.concat([test.drop(columns = ['sentiment', 'sentiment_label'], axis = 1), test_dummies], axis = 1)

In [10]:
train['char_count'] = train['text'].apply(lambda x: count_chars(x))
train['word_count'] = train['text'].apply(lambda x: count_words(x))
train['sent_count'] = train['text'].apply(lambda x: count_sent(x))
train['capital_char_count'] = train['text'].apply(lambda x: count_capital_chars(x))
train['capital_word_count'] = train['text'].apply(lambda x: count_capital_words(x))
# train['quoted_word_count'] = train['text'].apply(lambda x: count_words_in_quotes(x))
train['stopword_count'] = train['text'].apply(lambda x: count_stopwords(x))
train['unique_word_count'] = train['text'].apply(lambda x: count_unique_words(x))
                                                 
test['char_count'] = test['text'].apply(lambda x: count_chars(x))
test['word_count'] = test['text'].apply(lambda x: count_words(x))
test['sent_count'] = test['text'].apply(lambda x: count_sent(x))
test['capital_char_count'] = test['text'].apply(lambda x: count_capital_chars(x))
test['capital_word_count'] = test['text'].apply(lambda x: count_capital_words(x))
# test['quoted_word_count'] = test['text'].apply(lambda x: count_words_in_quotes(x))
test['stopword_count'] = test['text'].apply(lambda x: count_stopwords(x))
test['unique_word_count'] = test['text'].apply(lambda x: count_unique_words(x))
                                                 
## Average word length
train['avg_wordlength'] = train['char_count'] / train['word_count']
test['avg_wordlength'] = test['char_count'] / test['word_count']

## Average sentence lenght
train['avg_sentlength'] = train['word_count'] / train['sent_count']
test['avg_sentlength'] = test['word_count'] / test['sent_count']

## Unique words vs count words
train['unique_vs_words'] = train['unique_word_count'] / train['word_count']
test['unique_vs_words'] = test['unique_word_count'] / test['word_count']

## stopwords vs count words
train['stopwords_vs_words'] = train['stopword_count'] / train['word_count']
test['stopwords_vs_words'] = test['stopword_count'] / test['word_count']

In [11]:
vectorizer = TfidfVectorizer(max_features = 150)
# vectorizer = TfidfVectorizer()
train_docs = [train['clean_tweet'][i] for i in range(0, train.shape[0])]
test_docs = [test['clean_tweet'][i] for i in range(0, test.shape[0])]
train_tf_idf = vectorizer.fit_transform(train_docs).toarray()
test_tf_idf = vectorizer.transform(test_docs).toarray()

In [12]:
features_names = ['feature_' + str(i) for i in range(1, 151)]
train_tf_idf = pd.DataFrame(train_tf_idf, columns = features_names)
test_tf_idf = pd.DataFrame(test_tf_idf, columns = features_names)

In [13]:
train = pd.concat([train, train_tf_idf], axis = 1)
test = pd.concat([test, test_tf_idf], axis = 1)

In [14]:
train.to_csv('train_2.csv', index = False)
test.to_csv('test_2.csv', index = False)

In [ ]:
# train['char_count'] = train['clean_tweet'].apply(lambda x: count_chars(x))
# train['word_count'] = train['clean_tweet'].apply(lambda x: count_words(x))
# train['unique_word_count'] = train['clean_tweet'].apply(lambda x: count_unique_words(x))
                                                 
# test['char_count'] = test['clean_tweet'].apply(lambda x: count_chars(x))
# test['word_count'] = test['clean_tweet'].apply(lambda x: count_words(x))
# test['unique_word_count'] = test['clean_tweet'].apply(lambda x: count_unique_words(x))
                                                 
# ## Average word length
# train['avg_wordlength'] = train['char_count'] / train['word_count']
# test['avg_wordlength'] = test['char_count'] / test['word_count']

# ## Unique words vs count words
# train['unique_vs_words'] = train['unique_word_count'] / train['word_count']
# test['unique_vs_words'] = test['unique_word_count'] / test['word_count']

In [11]:
train.head()

,id,keyword,location,text,target,clean_tweet,subjectivity,negative,neutral,positive,char_count,word_count,sent_count,capital_char_count,capital_word_count,stopword_count,unique_word_count,avg_wordlength,avg_sentlength,unique_vs_words,stopwords_vs_words
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds reason may allah forgive us,0.0,0,1,0,69,13,1,10,1,5,13,5.307692,13.0,1.000000,0.384615
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,0.4,0,0,1,38,7,2,5,0,0,7,5.428571,3.5,1.000000,0.000000
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,residents asked shelter place notified officer...,0.4,1,0,0,133,22,2,2,0,9,20,6.045455,11.0,0.909091,0.409091
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13 000 people receive evacuation orders califo...,0.0,0,1,0,65,8,1,1,0,1,8,8.125000,8.0,1.000000,0.125000
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby smoke pours school,0.0,0,1,0,88,16,1,3,0,6,15,5.500000,16.0,0.937500,0.375000


In [12]:
train.to_csv('train.csv', index = False)
test.to_csv('test.csv', index = False)

In [ ]:
train['target'].value_counts() / train.shape[0]

In [ ]:
pd.crosstab(train['target'], train['sentiment_label'])